In [11]:
import os
import time
import requests
from ddgs import DDGS


def download_ddg_images(query: str, out_dir: str, n: int = 60, timeout: int = 5):
    os.makedirs(out_dir, exist_ok=True)

    # Znajdź najwyższy istniejący numer pliku
    existing = [f for f in os.listdir(out_dir) if f.lower().endswith((".jpg", ".jpeg", ".png"))]

    if existing:
        # wyciągnij numery np. 0001.jpg → 1
        nums = []
        for f in existing:
            try:
                nums.append(int(os.path.splitext(f)[0]))
            except ValueError:
                continue
        start_index = max(nums) + 1 if nums else 0
    else:
        start_index = 0

    idx = start_index
    saved = 0

    with DDGS() as ddgs:
        for r in ddgs.images(query, max_results=n):
            url = r.get("image")
            if not url:
                continue

            fname = f"{idx:06d}.jpg"      # 6 cyfr → nie będzie kolizji
            fpath = os.path.join(out_dir, fname)

            try:
                resp = requests.get(url, timeout=timeout)
                resp.raise_for_status()

                with open(fpath, "wb") as f:
                    f.write(resp.content)

                idx += 1
                saved += 1

                if saved >= n:
                    break

                time.sleep(0.5)
            except Exception:
                continue

    print(f"Pobrano {saved} obrazów do: {out_dir}. Ostatni index = {idx-1}")

In [17]:
download_ddg_images(
    query="historia Fiat 126p zdjęcia",
    out_dir="data/fiat126p_raw",
    n=60
)

Pobrano 59 obrazów do: data/fiat126p_raw. Ostatni index = 443


In [12]:
download_ddg_images("Fiat 126p black and white photo", "data/fiat126p_raw", n=40)
download_ddg_images("Fiat 126 Maluch PRL vintage photo ", "data/fiat126p_raw", n=40)
download_ddg_images("126p Polska ulica zima", "data/fiat126p_raw", n=40)
download_ddg_images("126p Poland 90s", "data/fiat126p_raw", n=40)

Pobrano 36 obrazów do: data/fiat126p_raw. Ostatni index = 113
Pobrano 39 obrazów do: data/fiat126p_raw. Ostatni index = 152
Pobrano 40 obrazów do: data/fiat126p_raw. Ostatni index = 192
Pobrano 40 obrazów do: data/fiat126p_raw. Ostatni index = 232
